# Questo notebook importa le raccomandazioni degli analisti da marketscreener, di default il link è di intesa san paolo ma può essere cambiato come riportato sotto

In [ ]:
#mounts current gdrive in colab after providing success code
#INSERISCI IL CODICE DI SUCCESSO O NON FUNZIONERA'
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/My Drive/INVEST-IO/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/INVEST-IO


In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
from selenium import webdriver
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import json
import pprint
import numpy as np

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

     |████████████████████████████████| 911kB 8.0MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:6 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:12 htt

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: use options instead of chrome_options


In [ ]:
def get_tds():
  wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
  #loop sulle raccomandazioni dalla pagina principale
  for i in wd.find_elements_by_css_selector('table#ALNI4.tabBody >* tr'):
    date = i.find_elements_by_css_selector('.pleft5.pright5.large30')[0].text
    reccom = i.find_elements_by_css_selector('.newsColCT.ptop3')[0].text
    link = i.find_elements_by_css_selector('.newsColCT.ptop3 > a')[0].get_attribute('href')
    output.append({'date': date, 'reccom' : reccom , 'link' : link})

In [ ]:
def click_on_button(x):
  element = wd.find_element_by_css_selector(x)
  wd.execute_script("arguments[0].click();", element)

In [ ]:
#MAIN
url = 'https://www.marketscreener.com/INTESA-SANPAOLO-SPA-68944/news-broker-research/' #< sostituisci qui il link
wd.set_window_size(1920,1080)
wd.maximize_window()
wd.get(url)

output = []
next_button = 1

while next_button > 0:
  get_tds()
  try:
    button_container = 'a.nPageEndTab'
    wd.find_element_by_css_selector(button_container)
    click_on_button(button_container)
  except:
    next_button = 0

In [ ]:
full_output = []
import time
def loop_single_reccomm(k):
  url = k['link']
  date = k['date']
  title = k['reccom']
  wd.get(url)
  time.sleep(abs(np.random.normal(2,1)))
  time_published = wd.find_element_by_css_selector('meta[itemprop="datePublished"]').get_attribute('content')
  desc = wd.find_element_by_id('grantexto').text
  full_output.append({'date_short' : date, 'title' : title, 'url': url, 'date_long' : time_published, 'desc' : desc})
  return full_output

for i in tqdm(output):
  loop_single_reccomm(i)
print (full_output)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



[{'date_short': '08/07', 'title': 'INTESA SANPAOLO S.P.A. : Upgraded to Buy by UBS', 'url': 'https://www.marketscreener.com/INTESA-SANPAOLO-SPA-68944/news/INTESA-SANPAOLO-S-P-A-Upgraded-to-Buy-by-UBS-31076623/', 'date_long': '2020-08-07T02:29:03-04:00', 'desc': "UBS's analyst Ignacio Cerezo upgrades his rating from Neutral to Buy. The target price has been lifted and is now set at EUR 2.40 compared to EUR 1.95 before."}, {'date_short': '08/05', 'title': 'INTESA SANPAOLO S.P.A. : Kepler Cheuvreux reaffirms its Buy rating', 'url': 'https://www.marketscreener.com/INTESA-SANPAOLO-SPA-68944/news/INTESA-SANPAOLO-S-P-A-Kepler-Cheuvreux-reaffirms-its-Buy-rating-31061006/', 'date_long': '2020-08-05T08:10:03-04:00', 'desc': 'Anna Maria Benassi from Kepler Cheuvreux retains his positive opinion on the stock with a Buy rating. The target price is reviewed upwards from EUR 2.10 to EUR 2.22.'}, {'date_short': '08/04', 'title': 'INTESA SANPAOLO S.P.A. : Jefferies reiterates its Buy rating', 'url': '

In [3]:
print (full_output)

NameError: ignored

In [2]:
from pandas import DataFrame
dataset = DataFrame (full_output)
from tqdm import tqdm # This is an awesome package for tracking for loops
import pandas as pd

df.to_csv('sr_market_screener_reccom.csv')

NameError: ignored

In [ ]:
import importlib
from textblob import TextBlob
# re = python module for regular expression
import re
from datetime import datetime, timedelta
from google.colab import files

# # # # TWITTER STREAM LISTENER - Analyzer # # # #
# we can use TextBlob for tweet sentiment analysis and determine the polarity of tweet, from -3 to 3;
# -3 = negative, 0 = neutral, 3 = positive
desc=data['desc']
desc_analyzer = DescAnalyzer() 
class DescAnalyzer():
    """
    Functionality for analyzing and categorizing content from tweets.
    """
    # claen_tweet using the re module to clean tweets text 
    def clean_desc(self, desc):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", desc).split())
    def analyze_sentiment(self, desc):
        analysis = TextBlob(self.clean_desc(desc))
        
        if analysis.sentiment.polarity > 0:
            return 3
        elif analysis.sentiment.polarity == 0:
            return 0
        else:
            return -3

#add the new column 'sentiment' by passing the class TwitterAnalyzer

data['sentiment'] = np.array([desc_analyzer.analyze_sentiment(desc) for desc in data['desc']])

data.head()

data.to_csv('sr_market_screener_reccom.csv')
files.download('sr_market_screener_reccom.csv') 